get best model of the job and compared the best two

In [1]:
job_name = 'sweet_bell_qc73gvv9m6'

In [2]:
import os
import urllib
from IPython.display import Image
from zipfile import ZipFile
from azure.ai.ml.entities import AmlCompute
from azure.core.exceptions import ResourceNotFoundError

from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from azure.ai.ml import MLClient
from azure.ai.ml import automl
from azure.identity import DefaultAzureCredential

from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes, InputOutputModes
from azure.ai.ml import Input

from PIL import Image
import jsonlines

import json
import mlflow
from mlflow.tracking.client import MlflowClient

def get_azure_credential(subscription_id, resource_group, workspace):
    credential = DefaultAzureCredential()
    ml_client = MLClient.from_config(credential)
    ml_client =  MLClient(credential, subscription_id, resource_group, workspace)
    return ml_client

ml_client = get_azure_credential("","","")



Found the config file in: /config.json


In [3]:

MLFLOW_TRACKING_URI = ml_client.workspaces.get(name=ml_client.workspace_name).mlflow_tracking_uri
print(MLFLOW_TRACKING_URI)

azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/46169265-43c5-42f4-b171-b27bdd8e5afa/resourceGroups/rchoque/providers/Microsoft.MachineLearningServices/workspaces/ws_ml_rcf


In [4]:


mlflow_client = MlflowClient()

In [5]:
mlflow_parent_run =  mlflow_client.get_run(job_name)
print(mlflow_parent_run)

<Run: data=<RunData: metrics={'AUC_macro': 1.0,
 'AUC_micro': 1.0,
 'AUC_weighted': 1.0,
 'accuracy': 1.0,
 'accuracy_train': 0.98961,
 'average_precision_score_macro': 1.0,
 'average_precision_score_micro': 1.0,
 'average_precision_score_weighted': 1.0,
 'f1_score_macro': 1.0,
 'f1_score_micro': 1.0,
 'f1_score_weighted': 1.0,
 'log_loss': 0.00255,
 'log_loss_train': 0.02697,
 'precision_score_macro': 1.0,
 'precision_score_micro': 1.0,
 'precision_score_weighted': 1.0,
 'recall_score_macro': 1.0,
 'recall_score_micro': 1.0,
 'recall_score_weighted': 1.0}, params={}, tags={'automl_best_child_run_id': 'sweet_bell_qc73gvv9m6_HD_3',
 'exp-name': 'Job for image classification with lemon objetcs',
 'fit_time_000': 'NaN',
 'is_gpu': 'True',
 'iteration_000': '0',
 'mlflow.rootRunId': 'sweet_bell_qc73gvv9m6',
 'mlflow.runName': 'sweet_bell_qc73gvv9m6',
 'mlflow.user': 'Ronald Choque',
 'model_explain_run': 'best_run',
 'pipeline_id_000': 'UnkownPipelineId',
 'predicted_cost_000': '0',
 'run_

In [6]:
print(mlflow_parent_run.data.tags['automl_best_child_run_id'])

sweet_bell_qc73gvv9m6_HD_3


In [7]:
best_child_run_id = mlflow_parent_run.data.tags['automl_best_child_run_id']
best_run = mlflow_client.get_run(best_child_run_id)

**id for create the endpoint and model**

In [8]:
best_run.info.run_id

'sweet_bell_qc73gvv9m6_HD_3'

**Metrics best Model**

In [9]:
best_run.data.metrics

{'accuracy_train': 0.98961,
 'log_loss_train': 0.02697,
 'log_loss': 0.00255,
 'AUC_macro': 1.0,
 'AUC_micro': 1.0,
 'f1_score_micro': 1.0,
 'average_precision_score_micro': 1.0,
 'AUC_weighted': 1.0,
 'average_precision_score_weighted': 1.0,
 'recall_score_micro': 1.0,
 'accuracy': 1.0,
 'average_precision_score_macro': 1.0,
 'precision_score_macro': 1.0,
 'recall_score_weighted': 1.0,
 'f1_score_macro': 1.0,
 'f1_score_weighted': 1.0,
 'recall_score_macro': 1.0,
 'precision_score_micro': 1.0,
 'precision_score_weighted': 1.0}

**descargar el modelo un folder local**

In [10]:

import os

local_dir = "./artifact_downloads"
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

In [11]:
local_path = mlflow_client.download_artifacts(best_run.info.run_id, "outputs", local_dir)
local_path

2023/11/29 14:57:57 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


'/mnt/batch/tasks/shared/LS_root/mounts/clusters/testnotebookrcf/code/Users/ron.todo.poderoso/Proyecto/artifact_downloads/outputs'

**TODO get metrics from the other algorith**